<a href="https://colab.research.google.com/github/kuroneko913/lab/blob/master/Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Pythonでブロックチェーンを実装して採掘までやってみたので解説する](https://paiza.hatenablog.com/entry/2018/05/11/Python%E3%81%A7%E3%83%96%E3%83%AD%E3%83%83%E3%82%AF%E3%83%81%E3%82%A7%E3%83%BC%E3%83%B3%E3%82%92%E5%AE%9F%E8%A3%85%E3%81%97%E3%81%A6%E6%8E%A1%E6%8E%98%E3%81%BE%E3%81%A7%E3%82%84%E3%81%A3%E3%81%A6)
を参考にブロックチェーンを作って取引記録をしてみた。

In [153]:
import json
import hashlib
from datetime import datetime

class Block:
    def __init__(self, index, timestamp, prev_hash, transaction):
        self.index = index
        self.timestamp = timestamp
        self.prev_hash = prev_hash
        self.transaction = [transaction]
        self.diff = 5 #難易度
        self.nonce = None # 採掘時に計算する対象 nonce
        self.now_hash = self.calc_hash()

    def calc_hash(self):
        joined_data = {
            'index' : self.index,
            'timestamp' : self.timestamp,
            'prev_hash' : self.prev_hash,
            'transaction' : self.transaction,
            'diff' : self.diff
        }
        json_text = json.dumps(joined_data, sort_keys = True)
        return hashlib.sha256(json_text.encode('ascii')).hexdigest()

    def __repr__(self) -> str:
        return f"""
            'index' : {self.index},
            'timestamp' : {self.timestamp},
            'prev_hash' : {self.prev_hash},
            'transaction' : {self.transaction},
            'diff' : {self.diff},
            'now_hash' : {self.now_hash},
            'nonce' : {self.nonce}
        """

    def check(self, nonce):
        nonce_joined = self.now_hash + str(nonce)
        calced = hashlib.sha256(nonce_joined.encode('ascii')).hexdigest()
        if calced[:self.diff:].count('0') == self.diff :
            return True
        return False

    def mining(self, append_transaction):
        nonce = 0
        # 報酬(好きな取引)
        self.transaction.append(append_transaction)
        self.now_hash = self.calc_hash()

        while True:
            nonce_joined = self.now_hash + str(nonce)
            calced = hashlib.sha256(nonce_joined.encode('ascii')).hexdigest()
            if calced[:self.diff:].count('0') == self.diff:
                break
            nonce += 1
        return nonce
    

In [154]:
class BlockChain:
    def __init__(self):
        self.chain = []
        self.prev_hash = self.calc_hash('initial_hash')
        self.index = 0

    def __len__(self):
        return len(self.chain)

    def __repr__(self):
        last_block = self.chain[-1] 
        return str([last_block.index, last_block.timestamp, last_block.prev_hash,last_block.transaction,last_block.now_hash, last_block.diff, last_block.nonce])

    def __getitem__(self, index):
        return self.chain[index]

    def append(self, transaction, append_transaction = None):
        timestamp = str(datetime.now())
        block = Block(self.index, timestamp, self.prev_hash, transaction)
        nonce = block.mining(append_transaction)
        if block.check(nonce) is True:
            block.nonce = nonce
            self.chain.append(block)
            self.prev_hash = block.now_hash
            self.index += 1 
            return self.prev_hash
        return False

    def calc_hash(self, value):
        json_text = json.dumps(value, sort_keys = True)
        return hashlib.sha256(json_text.encode('ascii')).hexdigest()

In [155]:
class Wallet:
    def __init__(self, name, amount=0):
        self.name = name
        self.amount = amount
        self.address = hashlib.sha256(name.encode('ascii')).hexdigest()

    def __repr__(self) -> str:
        return f"{self.name} {self.address} {self.amount}"

    # 入金(履歴に残らないので直接は呼ばない)
    def add(self, amount):
        self.amount += amount

    # 出金(履歴に残らないので直接は呼ばない)
    def remove(self, amount):
        if (self.amount - amount < 0):
            raise Exception('too little amount')
        self.amount -= amount

    def check_removable(self, amount):
        return self.amount - amount > 0
    
    # 送金
    '''
    deposit_flag is True => Walletへの入金とみなす
    '''
    def send(self, to_wallet = None, deposit_flag = False, amount = 0):
        if (to_wallet is None):
            raise Exception('sent wallet not found')
        if (deposit_flag is True):
            self.add(amount)
            return self.get_last_transaction(None, self, amount)
        if (self.amount - amount < 0):
            raise Exception('too little amount')
        self.remove(amount)
        to_wallet.add(amount)
        return self.get_last_transaction(self, to_wallet, amount)

    def get_last_transaction(self, from_wallet, to_wallet, amount):
        from_wallet_address = 'None' if from_wallet is None else from_wallet.address 
        return {'amount': amount, 'from': from_wallet_address, 'to': to_wallet.address, 'date': str(datetime.now())}


In [156]:
class Transaction:
    def __init__(self, block_chain:BlockChain):
        self.block_chain = block_chain
        self.reward_rate = 0.01 

    # nonce計算の報酬として取引額の1%を報酬受取用のWalletに送金する
    def exec(self, from_wallet:Wallet=None, to_wallet:Wallet=None, amount = 0, reward_wallet = None):
        if (to_wallet is None):
            raise Exception('送金先を指定してください')
        # Walletへの入金
        deposit_flag = True if from_wallet is None else False
        return self.make_transaction_with_rewards(from_wallet, to_wallet, deposit_flag, reward_wallet, amount)

    def __repr__(self):
        if (len(self.block_chain) == 0):
            return 'no transaction'
        return str(self.block_chain)

    def make_transaction_with_rewards(self, from_wallet, to_wallet, deposit_flag, reward_wallet, amount):
        if (deposit_flag is True):
            # 入金時は手数料を差し引いた金額が入金される
            payer_wallet = to_wallet 
        else: 
            amount_with_rewards = amount * (1 + self.reward_rate)
            payer_wallet = from_wallet
        # 手数料を支払うwalletに手数料含め支払い能力があるのかを確認 
        payer_wallet = from_wallet if (deposit_flag is False) else to_wallet
        if (deposit_flag is False and self.ability_to_pay_amounts(amount_with_rewards, payer_wallet) is False):
            return False
        if (deposit_flag is True):
            transaction_data = to_wallet.send(to_wallet, True, (1 - self.reward_rate) * amount)
        else:
            transaction_data = from_wallet.send(to_wallet, False, amount)
        reward_transaction_data = self.reward_transaction(reward_wallet, to_wallet, amount)
        self.block_chain.append(transaction_data, reward_transaction_data)
        return (transaction_data, reward_transaction_data)

    def ability_to_pay_amounts(self, amount, payer_wallet):
        return payer_wallet.check_removable(amount)

    # 送金額のx%を報酬として採掘者のWalletに送金する取引の実施
    def reward_transaction(self, reward_wallet, from_wallet, amount):
        return from_wallet.send(reward_wallet, False, amount * self.reward_rate)


In [157]:
wa = Wallet('A')
wb = Wallet('B')
wc = Wallet('C')
miner1 = Wallet('Miner1')

block = BlockChain()
transaction = Transaction(block)
transaction_data_list = [
    [None, wa, 10000, miner1],
    [None, wb, 10000, miner1],
    [None, wc, 10000, miner1],
    [wa, wb, 5000, miner1],
    [wb, wc, 5000, miner1],
    [wc, wa, 2500, miner1],
    [wc, wb, 1000, miner1],
]
transaction.exec(None, wc, 10000, miner1)

for data in transaction_data_list:
    result = transaction.exec(data[0], data[1], data[2], data[3])
    print(result)
    print(f"from: {data[0]}")
    print(f"to: {data[1]}")
    print(f"amount: {data[2]}")
    print(f"reward total: {data[3]}\n")


({'amount': 9900.0, 'from': 'None', 'to': '559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd', 'date': '2022-12-08 15:55:22.174453'}, {'amount': 100.0, 'from': '559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd', 'to': '365b1c7f02373cd936aa3ea58f250b7d59bf1c54eb0359f1f0bdf7f0e917c92f', 'date': '2022-12-08 15:55:22.174489'})
from: None
to: A 559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd 9800.0
amount: 10000
reward total: Miner1 365b1c7f02373cd936aa3ea58f250b7d59bf1c54eb0359f1f0bdf7f0e917c92f 200.0

({'amount': 9900.0, 'from': 'None', 'to': 'df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c', 'date': '2022-12-08 15:55:23.107889'}, {'amount': 100.0, 'from': 'df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c', 'to': '365b1c7f02373cd936aa3ea58f250b7d59bf1c54eb0359f1f0bdf7f0e917c92f', 'date': '2022-12-08 15:55:23.107921'})
from: None
to: B df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c 9800.0
amo

In [158]:
print(wa)
print(wb)
print(wc)
print(miner1)
print("")

for block in transaction.block_chain:
    # ブロック
    print(block)
    nonce_joined = block.now_hash+str(block.nonce)
    calced = hashlib.sha256(nonce_joined.encode('ascii')).hexdigest()
    # ルールを満たしているかの検証
    print(f"{calced[:block.diff:].count('0') == block.diff}, {calced}\n")
        
print("\n")

A 559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd 7275.0
B df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c 10740.0
C 6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d 21050.0
Miner1 365b1c7f02373cd936aa3ea58f250b7d59bf1c54eb0359f1f0bdf7f0e917c92f 535.0


            'index' : 0,
            'timestamp' : 2022-12-08 15:55:22.055871,
            'prev_hash' : 6bb51228573f8563d30efc73e1261754e5f29f881b5fa93ec7ce5300c8cbc9c4,
            'transaction' : [{'amount': 9900.0, 'from': 'None', 'to': '6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d', 'date': '2022-12-08 15:55:22.055845'}, {'amount': 100.0, 'from': '6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d', 'to': '365b1c7f02373cd936aa3ea58f250b7d59bf1c54eb0359f1f0bdf7f0e917c92f', 'date': '2022-12-08 15:55:22.055866'}],
            'diff' : 5,
            'now_hash' : a666fdb14ef2d84f28acc4f7618f88c05fd166e3cc8056adc55694a35ec7316d,
            'nonce' : 658

In [163]:
import random
wa = Wallet('A')
wb = Wallet('B')
wc = Wallet('C')
miner = Wallet('Miner1')

block = BlockChain()
transaction = Transaction(block)
for i in range(100):
    amount = random.randrange(10000)
    to_wallet = random.choice([wa, wb, wc])
    result = transaction.exec(None, to_wallet, amount, miner)
    block = transaction.block_chain[-1]
    # ブロック
    print(block)
    nonce_joined = block.now_hash+str(block.nonce)
    calced = hashlib.sha256(nonce_joined.encode('ascii')).hexdigest()
    # ルールを満たしているかの検証
    if (calced[:block.diff:].count('0') == block.diff):
        print(f"{calced[:block.diff:].count('0') == block.diff}, {calced}\n")

        print(result)
        print(f"to: {to_wallet}")
        print(f"amount: {amount}")
        print(f"reward total: {miner}\n")
    else:
        print("異常を検知")
        del(transaction.block_chain[-1])


            'index' : 0,
            'timestamp' : 2022-12-08 16:19:34.978447,
            'prev_hash' : 6bb51228573f8563d30efc73e1261754e5f29f881b5fa93ec7ce5300c8cbc9c4,
            'transaction' : [{'amount': 4601.5199999999995, 'from': 'None', 'to': '6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d', 'date': '2022-12-08 16:19:34.978410'}, {'amount': 46.480000000000004, 'from': '6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d', 'to': '365b1c7f02373cd936aa3ea58f250b7d59bf1c54eb0359f1f0bdf7f0e917c92f', 'date': '2022-12-08 16:19:34.978440'}],
            'diff' : 5,
            'now_hash' : 54cab71a0be0fe4cdf34fa24349f1619cca01625c8a725bcf2b0436f6410ca30,
            'nonce' : 3086914
        
True, 000002bfe1c291be2864d4d833e488663483d72ba504b9b4d95d6579bbfbb552

({'amount': 4601.5199999999995, 'from': 'None', 'to': '6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d', 'date': '2022-12-08 16:19:34.978410'}, {'amount': 46.480000000000004, 